In [ ]:
from SARLens.utils.io import dump

import pandas as pd
import numpy as np

import s1isp 
from s1isp.decoder import decoded_subcomm_to_dict
from s1isp.decoder import EUdfDecodingMode

def extract_echo_bursts(records):
    """
    Splits a list into sublists based on given indexes.

    Parameters:
        records: The list of records from isp.
    Returns:
    list of list: A list containing the sublists created by splitting the original list at the given indexes.
    """
    
    
    signal_types = {'noise': 1, 'tx_cal': 8, 'echo': 0}
    filtered = [x for x in isp['records'] if x[1].radar_configuration_support.ses.signal_type == signal_types['echo']]
    # Get number of quads 
    get_nq = lambda x: x[1].radar_sample_count.number_of_quads
    # Computing first and last number of quads:
    # In stripmap you only have two bursts 
    first_nq = get_nq(filtered[0])
    last_nq = get_nq(filtered[-1])
    # Creating list of number of quads:
    nqs_list = [first_nq, last_nq]
    # Filtering the bursts
    bursts = {idx: [x for x in filtered if get_nq(x) in [nq]] for idx, nq in enumerate(nqs_list)}
    return bursts


In [ ]:
filename = '/Users/robertodelprete/Desktop/AutoFocusNet/Data/DATA/S1A_S3_RAW__0SDH_20240524T213606_20240524T213631_054018_069139_241A.SAFE/s1a-s3-raw-s-hv-20240524t213606-20240524t213631-054018-069139.dat'

In [ ]:
records, offsets, subcom_data_records = s1isp.decoder.decode_stream(
    filename,
    # maxcount=6000,  # comment out this line to decode all the ISPs in the file
    udf_decoding_mode=EUdfDecodingMode.DECODE, # to have the corresponding signal data
)

In [ ]:
# isp = pd.read_pickle('s1a-s3-raw-s-hv-20240524t213606-20240524t213631-054018-069139.pkl')
# isp.keys() # 'records', 'offsets', 'subcomm_data_records'
isp = {'records':records, 'offsets':offsets, 'subcomm_data_records':subcom_data_records}

## To DataFrame

In [ ]:
headers_data = s1isp.decoder.decoded_stream_to_dict(isp['records'])
headers_data_df = pd.DataFrame(headers_data)

In [ ]:
subcom_data = isp['subcomm_data_records']
subcom_data_decoded = s1isp.decoder.SubCommutatedDataDecoder().decode(subcom_data)
subcom_data_decoded_dict = decoded_subcomm_to_dict(subcom_decoded=subcom_data_decoded)
subcom_data_decoded_df = pd.DataFrame(subcom_data_decoded_dict)

In [ ]:
headers_data_df

In [ ]:
bursts = extract_echo_bursts(records=records)

In [ ]:
get_echo_arr = lambda x: x.udf

burst_arr = np.array([get_echo_arr(x) for x in bursts[0]])
burst_arr